#### 1. Import libraries

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git@main 
!pip install evaluate peft

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, WeightedRandomSampler

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification 
from transformers import get_cosine_schedule_with_warmup

import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, accuracy_score

from datasets import load_dataset
import evaluate

from tqdm.notebook import tqdm
from peft import PeftModelForSequenceClassification, get_peft_config
from peft import (
    LoraConfig,
    PeftType,
    get_peft_model
)

In [2]:
peft_type = PeftType.LORA
peft_config = LoraConfig(task_type="SEQ_CLS", inference_mode=False, r=4, lora_alpha=32, lora_dropout=0.1)

#### 2. Upload data and model

In [3]:
model_name_or_path = "ai-forever/rugpt3large_based_on_gpt2"

batch_size = 8
num_epochs = 3

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

device: cuda


In [4]:
if any(k in model_name_or_path for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"

tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path, padding_side=padding_side)
tokenizer.add_special_tokens({'pad_token': '[PAD]'}) # padding token

model = GPT2ForSequenceClassification.from_pretrained(model_name_or_path, num_labels = 2)
model.config.pad_token_id = model.config.eos_token_id
model.resize_token_embeddings(len(tokenizer)) # for medium, large

Some weights of the model checkpoint at ai-forever/rugpt3large_based_on_gpt2 were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at ai-forever/rugpt3large_based_on_gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(50258, 1536)

In [5]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 592896 || all params: 760894464 || trainable%: 0.07792092439247922


In [6]:
path_data = '/content/drive/MyDrive/ds/exps/LLM/'


dataset = load_dataset("csv", data_files={'train': path_data + 'data/train2000.csv', 
                                           'test': path_data + 'data/test500.csv',
                                           'val': path_data + 'data/val500.csv'})

  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
def lowercase_condition(example):
    return {"text": example["text"].lower()}

def tokenize_function(examples):
    outputs = tokenizer(examples["text"], truncation  = True, max_length = 256)
    return outputs

dataset = dataset.map(lowercase_condition)

tok_dataset = dataset.map(
    tokenize_function,
    remove_columns=["Unnamed: 0", 'text', 'Unnamed: 0.1']
)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [8]:
# calculating class weight
class_weights = torch.Tensor([1 - tok_dataset["train"]['labels'].count(0) / len(tok_dataset["train"]['labels']), \
1 - tok_dataset["train"]['labels'].count(1) / len(tok_dataset["train"]['labels'])]).to(device)

In [9]:
def collate_fn(examples):
    return tokenizer.pad(examples, return_tensors="pt", padding = True)

torch.manual_seed(42)
    
#train_dataloader = DataLoader(tok_dataset["train"], shuffle = True, collate_fn=collate_fn, batch_size=batch_size)
eval_dataloader = DataLoader(tok_dataset["val"], collate_fn=collate_fn, batch_size=batch_size)

In [10]:
# to use WeightedRandomSampler

example_weights = [class_weights[e] for e in tok_dataset["train"]['labels']]
sampler = WeightedRandomSampler(example_weights, len(tok_dataset["train"]['labels']))
train_dataloader = DataLoader(tok_dataset["train"], sampler = sampler, collate_fn=collate_fn, batch_size=batch_size)

#### 3. Train model

In [11]:
def metrics_sklearn(y_true, pred):

    pred = torch.cat(pred).cpu()

    softmax = nn.Softmax(dim=1)
    pred = softmax(pred)
    pred.tolist()
    y_predict = pred.argmax(dim=-1)

    acc = round(accuracy_score(y_true, y_predict), 3)
    f1 = round(f1_score(y_true, y_predict), 3)
    rec = round(recall_score(y_true, y_predict), 3)
    prec = round(precision_score(y_true, y_predict), 3)
    rocauc = round(roc_auc_score(y_true, pred[:,1]), 3)

    return acc, f1, rec, prec, rocauc, y_predict

In [12]:
print(device)
print('\n')
model.to(device)

cuda




PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50258, 1536)
        (wpe): Embedding(2048, 1536)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-23): 24 x GPT2Block(
            (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): MergedLinear(
                in_features=1536, out_features=4608, bias=True
                (lora_dropout): Dropout(p=0.1, inplace=False)
                (lora_A): Linear(in_features=1536, out_features=8, bias=False)
                (lora_B): Conv1d(8, 3072, kernel_size=(1,), stride=(1,), groups=2, bias=False)
              )
              (c_proj): Conv1D()
              (attn_dropout): Dropout(p=0.1, inplace=False)
              (resid_dropout): Dropout(p=0.1, inplace=False)
            )
            (ln_2): LayerNorm((1536,), eps=1e-05, 

In [14]:
lr = 3e-4

optimizer = AdamW(params=model.parameters(), lr=lr)
metric = evaluate.load('accuracy')
#cross_entropy_loss = nn.CrossEntropyLoss(weight = class_weights)cross_entropy_loss = nn.CrossEntropyLoss()

lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.1 * (len(train_dataloader) * num_epochs),
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [15]:
train_losses = []
val_losses = []

for epoch in range(num_epochs):

    model.train()
    for step, batch in enumerate(tqdm(train_dataloader)):

        batch.to(device)
        outputs = model(**batch)

        loss = cross_entropy_loss(outputs.logits, batch["labels"])
        train_losses.append(loss)
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()


    model.eval()
    y_pred = []

    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch.to(device)

        with torch.no_grad():
            outputs = model(**batch)

        loss = cross_entropy_loss(outputs.logits, batch["labels"])
        val_losses.append(loss)

        predictions = outputs.logits.argmax(dim=-1)
        predictions, references = predictions, batch["labels"]
        metric.add_batch(
            predictions=predictions,
            references=references,
        )
        y_pred.append(outputs.logits)

    eval_metric = metric.compute()
    print(f"epoch {epoch}:", eval_metric)

    acc, f1, rec, prec, rocauc, y_predict = metrics_sklearn(tok_dataset["val"]['labels'], y_pred)
    print(acc, f1, rec, prec, rocauc)

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

epoch 0: {'accuracy': 0.64}
0.64 0.702 0.848 0.599 0.743


  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

epoch 1: {'accuracy': 0.84}
0.84 0.847 0.884 0.812 0.91


  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

epoch 2: {'accuracy': 0.848}
0.848 0.858 0.916 0.806 0.931


In [16]:
model.print_trainable_parameters()

trainable params: 592896 || all params: 760894464 || trainable%: 0.07792092439247922


#### 4. Test evaluation

In [17]:
torch.manual_seed(42)
test_dataloader = DataLoader(tok_dataset["test"], shuffle = False, collate_fn=collate_fn, batch_size = 8)

In [18]:
y_pred = []

for step, batch in enumerate(tqdm(test_dataloader)):
  batch.to(device)

  with torch.no_grad():
    outputs = model(**batch)

  predictions = outputs.logits.argmax(dim=-1)
  y_pred.append(outputs.logits)

  0%|          | 0/63 [00:00<?, ?it/s]

In [19]:
acc, f1, rec, prec, rocauc, y_predict = metrics_sklearn(tok_dataset["test"]['labels'], y_pred)
#param = sum(p.numel() for p in model.parameters())

In [20]:
y_predict.tolist().count(0), y_predict.tolist().count(1)

(214, 286)

In [21]:
comment = 'unb, sampler, lora, r4, alpha32, gpu14.6, cpu6.2'
vers = 'large'
inf = '25c'
ft = '4m'
param = 592896

In [22]:
path = 'drive/MyDrive/ds/exps/LLM/'
res = pd.read_csv(path + 'res.csv')
res.loc[len(res)] = ['rugpt', vers, param, comment, num_epochs, inf, ft, acc, f1, rec, prec, rocauc, batch_size, lr]
res

,model,version,size,comment,epochs,time_inf,time_ft_1e,accuracy,f1,precision,recall,roc_auc,batch,lr
0,rugpt,small,125233152,"balanced data, 1000tr, 100te",3,1c,25c,0.830,0.828,0.820,0.837,0.895,8,0.0003
1,rugpt,medium,355874816,"balanced data, embed layer + 1",3,3c,1.20m,0.870,0.871,0.880,0.863,0.937,8,0.0003
2,rugpt,large,760304640,out of memory,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,0.0003
3,rugpt,small,125228544,"unbalanced data, 2000tr, 500te, cross-entropy ...",3,2c,53c,0.872,0.864,0.816,0.919,0.940,8,0.0003
4,rugpt,medium,355874816,unbalanced data,3,10c,2m47c,0.880,0.877,0.852,0.903,0.957,8,0.0003
5,rugpt,small,125228544,"unbalanced data, CELoss weighted",3,4c,57c,0.882,0.876,0.836,0.921,0.952,8,0.0003
6,rugpt,medium,355874816,"unbalanced data, CELoss weighted",3,11c,2m54c,0.872,0.866,0.828,0.908,0.953,8,0.0003
7,rugpt,small,125228544,"unb, sampler",3,3c,54c,0.858,0.850,0.804,0.901,0.950,8,0.0003
8,rugpt,medium,355874816,"unb, sampler",3,11c,2m50c,0.884,0.879,0.840,0.921,0.952,8,0.0003
9,rugpt,medium,355874816,"unb, without ft",3,11c,NaN,0.500,0.658,0.964,0.500,0.558,8,0.0003


In [ ]:
res.to_csv(path + 'res.csv',  index = False)

In [25]:
res[res.version == 'medium']

,model,version,size,comment,epochs,time_inf,time_ft_1e,accuracy,f1,precision,recall,roc_auc,batch,lr
1,rugpt,medium,355874816,"balanced data, embed layer + 1",3,3c,1.20m,0.870,0.871,0.880,0.863,0.937,8,0.0003
4,rugpt,medium,355874816,unbalanced data,3,10c,2m47c,0.880,0.877,0.852,0.903,0.957,8,0.0003
6,rugpt,medium,355874816,"unbalanced data, CELoss weighted",3,11c,2m54c,0.872,0.866,0.828,0.908,0.953,8,0.0003
8,rugpt,medium,355874816,"unb, sampler",3,11c,2m50c,0.884,0.879,0.840,0.921,0.952,8,0.0003
9,rugpt,medium,355874816,"unb, without ft",3,11c,NaN,0.500,0.658,0.964,0.500,0.558,8,0.0003
12,rugpt,medium,788480,"unb, cel-n, lora, r8, gpu13.7, cpu6.2",3,11c,2m,0.834,0.822,0.768,0.885,0.921,8,0.0003
13,rugpt,medium,788480,"unb, cel-w, lora, r8, gpu13.7, cpu6.2",3,11c,2m,0.854,0.857,0.872,0.842,0.922,8,0.0003
14,rugpt,medium,395264,"unb, cel-n, lora, r4, alpha32 gpu13.7, cpu6.2",3,11c,2m,0.852,0.846,0.812,0.883,0.934,8,0.0003
15,rugpt,medium,395264,"unb, cel-w, lora, r4, alpha32 gpu13.7, cpu6.2",3,12c,2m,0.862,0.862,0.864,0.861,0.936,8,0.0003
16,rugpt,medium,395264,"unb, sampler, lora, r4, alpha32, gpu13.7, cpu6.2",3,12c,2m,0.858,0.855,0.840,0.871,0.932,8,0.0003


In [28]:
#res[res['time_ft_1e'] == '40c']['size']

,model,version,size,comment,epochs,time_inf,time_ft_1e,accuracy,f1,precision,recall,roc_auc,batch,lr
11,rugpt,small,125523456,"unb, celw, lora, r 8, alpha 16",3,3c,40c,0.884,0.879,0.844,0.917,0.954,8,0.0003


#### 5. Conclusion

- Для Large model не хватает ресурсов. 
- Small модель показала наилучший ррезультат для weighted CELoss.
- Medium  - с использованием sampler.

#### 6. Extra calculations

In [ ]:
# how batched are built with sampler

arr_batch = []
for step, batch in enumerate(train_dataloader):
  arr_batch += batch['labels'].tolist()  

arr_batch.count(0), arr_batch.count(1)

(971, 1029)

In [ ]:
dataset['train']['labels'].count(0), dataset['train']['labels'].count(1)